# Просмотр коммитов с уязвимостями

In [4]:
import polars as pl
from const import GITHUB_COMMIT_DATA_PATH, GITHUB_CVES_DATA_PATH, CVE_PROJECT_CVES_DATA_PATH
from helpers import enrich_commits_data


In [22]:
commits = enrich_commits_data(pl.read_csv(GITHUB_COMMIT_DATA_PATH))
cve_data: pl.DataFrame = pl.concat(
    [
        pl.read_csv(GITHUB_CVES_DATA_PATH),
        pl.read_csv(CVE_PROJECT_CVES_DATA_PATH).select(["cve_id", "reference", "cwe_id"]),
    ]
)
commits = commits.join(cve_data.select("cve_id", "cwe_id"), on="cve_id")
commits.describe()

describe,repo,commit,file,status,additions,deletions,changes,patch,file_url,cve_id,year,file_extension,language,cwe_id
str,str,str,str,str,f64,f64,f64,str,str,str,f64,str,str,str
"""count""","""1679043""","""1679043""","""1679043""","""1679043""",1.679043e6,1.679043e6,1.679043e6,"""1679043""","""1679043""","""1679043""",1.679043e6,"""1679043""","""1679043""","""1679043"""
"""null_count""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,"""86838""","""4""","""0""",0.0,"""0""","""0""","""529893"""
"""mean""",null,null,null,null,46.809474,15.608341,62.417814,null,null,null,2019.810974,null,null,null
"""std""",null,null,null,null,551.359934,277.326709,712.147564,null,null,null,2.816054,null,null,null
"""min""","""00xc/xml-rs""","""000030feb7a30f…",""".DS_Store""","""added""",0.0,0.0,0.0,"""@@ -0,0 +1 @@ …","""https://github…","""1999-0199""",1999.0,"""0/CREDITS""","""0/CREDITS""","""CWE-1004"""
"""25%""",null,null,null,null,1.0,0.0,3.0,null,null,null,2018.0,null,null,null
"""50%""",null,null,null,null,6.0,1.0,10.0,null,null,null,2020.0,null,null,null
"""75%""",null,null,null,null,21.0,5.0,32.0,null,null,null,2022.0,null,null,null
"""max""","""zzxiang/extern…","""v4""","""zproject/urls.…","""renamed""",107422.0,101208.0,208630.0,"""@@ -999,7 +999…","""https://github…","""2023-7076""",2023.0,"""zsh-theme""","""zsh-theme""","""CWE-99"""


In [26]:
# Фильтрация коммитов связанных с python
python_cves = set(commits.filter(pl.col("language") == "Python").select(pl.col("cve_id")).unique("cve_id").to_series())
python_commits = commits.filter(pl.col("cve_id").is_in(python_cves), pl.col("cwe_id") == "CWE-79", pl.col("commit") != "master")
with pl.Config(fmt_str_lengths=200, tbl_width_chars=150, tbl_rows=20):
    print(
        python_commits.select(["repo", "commit", "cve_id", "cwe_id"])
        .unique()
        .select(pl.format("https://github.com/{}/commit/{}", pl.col("repo"), pl.col("commit")), "cwe_id")
    )

shape: (166, 2)
┌────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────┐
│ literal                                                                                                ┆ cwe_id │
│ ---                                                                                                    ┆ ---    │
│ str                                                                                                    ┆ str    │
╞════════════════════════════════════════════════════════════════════════════════════════════════════════╪════════╡
│ https://github.com/ceph/ceph/commit/af3fffab3b0f13057134d96e5d481e400d8bfd27                           ┆ CWE-79 │
│ https://github.com/devsnd/cherrymusic/commit/62dec34a1ea0741400dd6b6c660d303dcd651e86                  ┆ CWE-79 │
│ https://github.com/salopensource/sal/commit/145bb72daf8460bdedbbc9fb708d346283e7a568                   ┆ CWE-79 │
│ https://github.com/liangliangyy/djangoblog/commit/c2bf